<a href="https://colab.research.google.com/github/bcmin1018/NLP/blob/main/%EA%B8%B0%EA%B3%84%EB%B2%88%EC%97%AD(rnn%2Battention)_%EC%BD%94%EB%93%9C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim
from nltk.tokenize import word_tokenize
import os
import nltk
import codecs
import csv
from konlpy.tag import Okt
import torch.nn.functional as F
import pandas as pd
nltk.download('punkt')

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 37.1 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os, sys 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 문어체_뉴스 다운로드
!gdown --id 1rRvQ9n52tXQgskDz_Q8F12zSlCedp9gG
# !gdown --id 1lXi-K8cs9gih6j3FyTKWLtapHh1JmWkD
# !gdown --id 1u9DAVk4bNjE8-qiLpUQRloFpjckV_KjO
# !gdown --id 1IpxnKROkWiLiJGOK3DoHuzX2oBEjC5ks

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1rRvQ9n52tXQgskDz_Q8F12zSlCedp9gG
To: /content/3_문어체_뉴스(1)_200226.xlsx
100% 51.4M/51.4M [00:00<00:00, 134MB/s]


In [3]:
# 문어체 뉴스 데이터의 파일 정보
train_data_path= "./3_문어체_뉴스(1)_200226.xlsx"

In [63]:
# train_data = pd.read_excel(train_data_path)
# train_data = train_data[['원문', '번역문']].values
train_data = train_data[['원문', '번역문']].values[0:100]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until


IndexError: ignored

In [17]:
class GRUMT(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, max_length, device, dropout_p=0.1):
        super(GRUMT, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.device = device

        # Encoder 부분
        self.encoder_embedding = nn.Embedding(input_size, hidden_size)
        self.encoder_rnn = nn.GRU(self.hidden_size, self.hidden_size)

        # Decoder 부분
        self.decoder_rnn = nn.GRU(self.hidden_size, self.hidden_size)
        self.decoder_embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.out = nn.Linear(self.hidden_size, self.output_size)

        self.loss = nn.NLLLoss()

    def _encoder(self, input_tensor, input_length):
        encoder_hidden = self._init_hidden()
        encoder_outputs = torch.zeros(self.max_length, self.hidden_size, device=self.device)

        for idx in range(input_length):
            input_tensor_step = input_tensor[idx]
            embedded = self.encoder_embedding(input_tensor_step).view(1, 1, -1)
            encoder_output, encoder_hidden = self.encoder_rnn(embedded, encoder_hidden)
            encoder_outputs[idx] = encoder_output[0, 0]

        return encoder_outputs, encoder_hidden

    def _decoder(self, target_tensor, target_length, encoder_hidden, encoder_outputs):
        decoder_input = torch.tensor([[SOS_token]], device=self.device)
        decoder_hidden = encoder_hidden

        loss_sum = 0
        for di in range(target_length):
            embedded = self.decoder_embedding(decoder_input).view(1, 1, -1)
            embedded = self.dropout(embedded)

            decoder_attention = F.softmax(self.attn(torch.cat((embedded[0], decoder_hidden[0]), 1)), dim=1)
            attn_applied = torch.bmm(decoder_attention.unsqueeze(0), encoder_outputs.unsqueeze(0))

            output = torch.cat((embedded[0], attn_applied[0]), 1)
            output = self.attn_combine(output).unsqueeze(0)

            output = F.relu(output)
            output, decoder_hidden = self.decoder_rnn(output, decoder_hidden)

            decoder_output = F.log_softmax(self.out(output[0]), dim=1)

            _, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            target_output = torch.tensor([target_tensor[di]], device=self.device)
            loss_sum += self.loss(decoder_output, target_output)
            if decoder_input.item() == EOS_token:
                break

        return loss_sum

    def forward(self, input_tensor, input_length, target_tensor, target_length):
        encoder_outputs, encoder_hidden = self._encoder(input_tensor, input_length)
        loss_sum = self._decoder(target_tensor, target_length, encoder_hidden, encoder_outputs)
        return loss_sum

    def _init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [96]:
def make_dic(data):
  eng_tokenizer = word_tokenize
  kor_tokenizer = Okt().morphs

  eng_dic = LangDic('en', eng_tokenizer)
  kor_dic = LangDic('ko', kor_tokenizer)

  for idx in range(0, len(data)):
    kor_sen = data[idx][0]
    eng_sen = data[idx][1]

    kor_dic.add_sentence(eng_sen)
    eng_dic.add_sentence(kor_sen)
    
  return eng_dic, kor_dic

def make_data_loader(data_pairs, eng_dic, kor_dic, max_len, batch_size):
    ds = MTDataset(data_pairs, eng_dic, kor_dic, max_len)
    return DataLoader(ds, batch_size=batch_size)

In [12]:
class MTDataset(Dataset):
    def __init__(self, data_pairs, eng_dic, kor_dic, max_len):
        super(MTDataset, self).__init__()

        self.max_len = max_len
        self.pair_data = list()

        for idx in range(0, len(data_pairs)):
          kor_sen = data_pairs[idx][0]
          eng_sen = data_pairs[idx][1]
          eng_sen_words, eng_sen_len = eng_dic.sentence2tensor(eng_sen, max_len)
          kor_sen_words, kor_sen_len = kor_dic.sentence2tensor(kor_sen, max_len)
          self.pair_data.append((eng_sen_words, eng_sen_len, kor_sen_words, kor_sen_len))

        self.data_len = len(self.pair_data)

    def __getitem__(self, idx):
        eng_sen_words, eng_sen_len, kor_sen_words, kor_sen_len = self.pair_data[idx]
        return eng_sen_words, eng_sen_len, kor_sen_words, kor_sen_len

    def __len__(self):
        return self.data_len

In [90]:
class LangDic:
    # 언어마다 단어 사전을 정의합니다.
    def __init__(self, name, tokenizer):
        # 클래스의 첫 시작인 함수입니다. 여기서 모델에 필요한 여러 변수들을 정의합니다.
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
        self.n_words = 2
        self.tokenizer = tokenizer

    def add_sentence(self, sentence):
        # 문장을 받아 문장에서 단어를 확인합니다.
        for word in self.tokenizer(sentence):
            self.add_word(word)

    def add_word(self, word):
        # 단어를 보고 그 단어가 사전에 존재하는지 아닌지를 살펴봅니다.
        # 존재하지 않는 경우 단어를 사전에 등록합니다.
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def sentence2tensor(self, sentence, max_len):
        # 가지고 있는 사전을 바탕으로 문장을 tensor의 형태로 바꿉니다.
        # tensor 내 들어있는 값은 단어 index이며 이를 통해 모델의 임베딩에 입력으로 줄 수 있습니다.
        indexes = list()
        for word in self.tokenizer(sentence):
          indexes.append(self.word2index[word])
            # try:
                # indexes.append(self.word2index[word])
            # except KeyError:
                # indexes.append(UNK_token)

        indexes.append(EOS_token)
        len_sen = len(indexes)
        if len_sen > max_len:
            indexes = indexes[:max_len-1]
            indexes.append(EOS_token)
            len_sen = max_len

        index_tensor = torch.tensor(indexes)
        return index_tensor, len_sen

In [19]:
def train(model, device, optimizer, train_loader, num_epochs):
    running_loss = 0.0
    best_loss = 0.0
    model.train()
    for epoch in range(num_epochs):
        for input_tensor, input_length, target_tensor, target_length in train_loader:
            input_tensor = input_tensor[0].to(device)
            target_tensor = target_tensor[0].to(device)

            loss_sum = model(input_tensor, input_length, target_tensor, target_length)
            optimizer.zero_grad()
            loss_sum.backward()
            optimizer.step()
            running_loss += loss_sum.item() / target_length.item()
            if running_loss < best_loss:
              print("best_loss updated")
              torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/model.pt')
              best_loss = running_loss
        print('Epoch {}, Train Loss: {:.4f}'.format(epoch + 1, running_loss))

In [ ]:
on_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0
EOS_token = 1
UNK_token = 2

max_len = 10
hidden_size = 256

eng_dic, kor_dic = make_dic(train_data)

eng_dic_size = eng_dic.n_words
kor_dic_size = kor_dic.n_words

train_loader = make_data_loader(train_data, eng_dic, kor_dic, max_len, 1)

In [ ]:
model = GRUMT(eng_dic_size, hidden_size, kor_dic_size, max_len, on_device).to(on_device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train(model, on_device, optimizer, train_loader, 20)

Epoch 1, Train Loss: 1839.3313
Epoch 2, Train Loss: 3290.3877
Epoch 3, Train Loss: 4689.4274


KeyboardInterrupt: ignored

## evaluate 함수

해당 함수에서는 validation data를 이용하여 학습된 `model`을 평가합니다.

In [ ]:
def evaluate(model, device, valid_loader):
    # 학습 중 모델을 평가합니다.
    # 모델에게 학습이 아닌 평가를 할 것이라고 알립니다.
    model.eval()
    valid_running_loss = 0.0

    # 학습이 아니기에 최적화를 하지 않는다는 환경을 설정합니다.
    with torch.no_grad():
        # validation 데이터를 읽습니다.
        for input_tensor, input_length, target_tensor, target_length in valid_loader:
            input_tensor = input_tensor[0].to(device)
            target_tensor = target_tensor[0].to(device)

            # model을 함수처럼 호출하면 model에서 정의한 forward 함수가 실행됩니다.
            # 즉, 데이터를 모델에 집어넣어 forward방향으로 흐른 후 그 결과를 받습니다.
            loss_sum = model(input_tensor, input_length, target_tensor, target_length)

            # validation 데이터의 loss, 즉 모델의 출력과 실제 데이터의 차이를 구합니다.
            valid_running_loss += loss_sum.item() / target_length.item()

    # 평균 loss를 계산하여 반환합니다.
    return valid_running_loss / len(valid_loader)

## 데이터 불러오기

데이터를 불러올 때 우리는 먼저 언어별 사전을 정의하고 그 사전에 맞춰 데이터를 불러옵니다.

- 문제 2. `valid_loader`를 불러오세요. 힌트) `train_loader`을 참고하세요.

In [ ]:
on_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# on_device = torch.device('cpu')

# 사전 내 특수 단어의 index를 각각 미리 정의합니다.
SOS_token = 0
EOS_token = 1
UNK_token = 2

# 데이터의 파일 정보
train_file_path = "./train_en_ko.csv"
valid_file_path = "./valid_en_ko.csv"

max_len = 10
hidden_size = 256

# 언어 별 사전 생성
eng_dic, kor_dic = make_dic(dataset_path=train_file_path)

#  train, validation 데이터 csv 파일을 읽어옵니다.
train_loader = make_data_loader(train_file_path, eng_dic, kor_dic, max_len, 1)

# <ToDo>: valid_dataset을 불러오세요.
valid_loader = make_data_loader(valid_file_path, eng_dic, kor_dic, max_len, 1)

# 영어와 한국어 사전의 크기(단어 개수)를 가져옵니다.
eng_dic_size = eng_dic.n_words
kor_dic_size = kor_dic.n_words

## 모델 학습

- 문제 3. `GRUMT` 인스턴스를 만드세요.
- 문제 4. `train` 함수를 이용하여 train data를 통해 모델 학습을 진행하세요.

In [ ]:
# <ToDo>: GRUMT 클래스의 인스턴스를 만드세요. 인스턴스 생성 시 필요한 parameter를 전달해주세요.
model = GRUMT(eng_dic_size, hidden_size, kor_dic_size, max_len, on_device).to(on_device)

In [ ]:
# Adam optimizier를 사용합니다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# <ToDo>: 학습을 위해 train 함수의 적절한 parameter를 전달해주세요.
train(model, on_device, optimizer, train_loader, valid_loader, 20)

Epoch 1, Step 100, Train Loss: 4.3022, Valid Loss: 3.3302
Epoch 1, Step 200, Train Loss: 3.8001, Valid Loss: 3.7074
Epoch 1, Step 300, Train Loss: 3.8735, Valid Loss: 3.5491
Epoch 1, Step 400, Train Loss: 4.2952, Valid Loss: 4.3542
Epoch 1, Step 500, Train Loss: 4.1805, Valid Loss: 4.7959
Epoch 1, Step 600, Train Loss: 4.0673, Valid Loss: 3.3506
Epoch 1, Step 700, Train Loss: 4.2876, Valid Loss: 4.0127
Epoch 1, Step 800, Train Loss: 4.2454, Valid Loss: 3.8350
Epoch 1, Step 900, Train Loss: 3.9232, Valid Loss: 3.2819
Epoch 1, Step 1000, Train Loss: 4.0964, Valid Loss: 2.6605
Epoch 1, Step 1100, Train Loss: 3.8758, Valid Loss: 4.3409
Epoch 1, Step 1200, Train Loss: 4.2298, Valid Loss: 3.4893
Epoch 1, Step 1300, Train Loss: 3.8367, Valid Loss: 2.6529
Epoch 1, Step 1400, Train Loss: 4.0012, Valid Loss: 3.4456
Epoch 1, Step 1500, Train Loss: 3.9890, Valid Loss: 3.0271
Epoch 1, Step 1600, Train Loss: 3.7667, Valid Loss: 3.1136
Epoch 1, Step 1700, Train Loss: 4.2531, Valid Loss: 4.5205
Epoch 

새로운 학습 데이터가 추가되었을 때 그 성능은 어떻게 변하는지.


다른 스타일의 테스트 데이터인 경우 그 성능은 어떻게 변하는지?

위의 질문들에서 문제점이 있다면 이를 어떻게 극복하면 좋을지?